In [ ]:
from classy import Class
import numpy as np
from matplotlib import pyplot as plt

Potential and derivatives

In [ ]:
from math import pi
csteGrav = 6.6743015e-11
M_pl = 1/np.sqrt(8*pi*csteGrav)

# M_pl = 1

def V_brane(phi,M4,mu,n):
    return(M4/(1+pow(mu/phi,n)))

def V_shift(phi,phi0,M4,mu,n):
    return(V_brane(phi0-phi,M4,mu,n))

def dV_brane(phi,M4,mu,n):
    factor = M4*n*pow(mu,n)
    denom = pow(phi,n)+pow(mu,n)
    return(factor*pow(phi,n-1)/pow(denom,2))

def ddV_brane(phi,M4,mu,n):
    factor = M4*n*pow(mu,n)
    factor2 = (n-1)*pow(mu,n)-(n+1)*pow(phi,n)
    denom = pow(mu,n)+pow(phi,n)
    return(factor*factor2/pow(denom,3))

Slow-roll parameters

In [ ]:

def epsilon1(phi,M4,mu,n):
    x=phi/mu
    factor=pow(n*M_pl/mu, 2)
    return( 0.5*factor/pow(x*(pow(x,n)+1),2)  )

def epsilon1_bis(phi,M4,mu,n):
    return(0.5*M_pl*pow(dV_brane(phi,M4,mu,n)/V_brane(phi,M4,mu,n),2))

def epsilon2(phi,M4,mu,n):
    x=phi/mu
    factor = 2*n*pow(M_pl/mu,2)
    numerator = (n+1)*pow(x,n)+1
    denominator = x*(pow(x,n)+1)
    return( factor*numerator/pow(denominator,2) )

def epsilon3(phi,M4,mu,n):
    x=phi/mu
    factor = n*pow(M_pl/mu,2)
    numerator = 2 + (n-4)*(n+1)*pow(x,n)  + (n+1)*(n+2)*pow(x,2*n)
    denominator = pow( x*(pow(x,n)+1) , 2)
    denominator = denominator* ((n+1)*pow(x,n) + 1 )
    return( factor * numerator / denominator)

In [ ]:
def epsilon1_true(phi,M4,mu,n):
    factor = 0.5*pow(M_pl/mu,2)
    return(factor*pow(dV_brane(phi,M4,mu,n)/V_brane(phi,M4,mu,n),2)  )

def epsilon2_true(phi,M4,mu,n):
    factor1 = pow(M_pl/mu,2)
    factor2 = 0.5*pow(M_pl/mu,2)
    quotient1 = ddV_brane(phi,M4,mu,n)/V_brane(phi,M4,mu,n)
    quotient2 = dV_brane(phi,M4,mu,n)/V_brane(phi,M4,mu,n)
    return(factor1*factor2*(pow(quotient2,2) - quotient1))

Calculate $\phi_{end}$ and $\phi_{start}$

In [ ]:
## précision nécessaire pour pouvoir atteindre phi tel que  V(phi) proche de M4   (on utilise cette précision dans la fonction calculate phi start)
def precision_mu(mu,N):
    return(pow(10,-(49-N*(np.log10(mu/M_pl)+3))))
    return(pow(10,-N))
    return(pow(10,10-np.log10(mu/M_pl)))
    return(pow(10,-(49-N*(np.log10(mu/M_pl)+3))))

In [ ]:
#Calcule phi tel que V(phi) proche de M4 ... la précision recquise est expliquée après  (en fait elle dépend de M4 et donc de mu  (car M4/mu4 = 16/27))
def calculate_phi_start_BI(M4,mu,n,precision):
    return(mu*pow(M4/precision,1/n))

In [ ]:
def calculate_phi_end_BI(M4,mu,n,precision):
    N0=1000
    phi_droite = calculate_phi_start_BI(M4,mu,n,precision)
    liste_phi = np.linspace(1e-50,phi_droite,N0)
    i_gauche = 0
    i_droite = N0-1

    i_mid = (i_gauche+i_droite)//2

    while (i_gauche-i_droite)<=2:
        print(i_gauche,i_droite)
        if epsilon1(liste_phi[i_mid],M4,mu,n)>1:
            i_gauche=i_mid
        else:
            i_droite=i_mid
        
        i_mid = (i_gauche+i_droite)//2
    return(liste_phi[i_mid])



In [ ]:
def essai_phi_end(N):
    n=4
    mu = 1e-2*M_pl
    M4=4*pi*pi*16/27
    eta=precision_mu(mu,N)
    calculate_phi_end_BI(M4,mu,n,eta)

essai_phi_end(4)

In [ ]:
from matplotlib.patches import Rectangle

def plot_BI(M4,mu,n,N):

    fig = plt.figure()
    ax1 = plt.subplot(3,3,1)
    ax2 = plt.subplot(3,3,4)
    ax3 = plt.subplot(3,3,5)
    ax4 = plt.subplot(3,3,6)
    ax5 = plt.subplot(3,3,7)
    ax6 = plt.subplot(3,3,8)
    ax7 = plt.subplot(3,3,9)

    eta = precision_mu(mu,N)
    phi_start = calculate_phi_start_BI(M4,mu,n,eta)
    print(r'\phi_{start}/M_pl}',phi_start)
    phi = np.linspace(1e-100,phi_start,10000)
    V = V_brane(phi,M4,mu,n)
    E1 = epsilon1(phi,M4,mu,n)
    E2 = epsilon2(phi,M4,mu,n)
    E3 = epsilon3(phi,M4,mu,n)
    E1bis = epsilon1_true(phi,M4,mu,n)
    E2bis = epsilon2_true(phi,M4,mu,n)

    ax1.plot(phi/mu,V/M4)
    ax2.plot(phi/mu,E1)
    ax3.plot(phi/mu,E2)
    ax4.plot(phi/mu,E3)
    ax5.plot(phi/mu,E1bis)
    ax6.plot(phi/mu,E2bis)
    # ax7.plot(phi/mu,E3bis)

    
    fig.suptitle(f'Brane Inflation Potential and slow-roll parameters for p = {n}') 

    ax1.set_xlabel(r'$\phi / \mu$')
    ax2.set_xlabel(r'$\phi / \mu$')
    ax3.set_xlabel(r'$\phi / \mu$')
    ax4.set_xlabel(r'$\phi / \mu$')
    ax5.set_xlabel(r'$\phi / \mu$')
    ax6.set_xlabel(r'$\phi / \mu$')
    ax7.set_xlabel(r'$\phi / \mu$')

    ax1.set_ylabel(r'$V(\phi)$')
    ax2.set_ylabel(r"$\epsilon_1(\phi)$")
    ax3.set_ylabel(r"$\epsilon_2(\phi)$")
    ax4.set_ylabel(r"$\epsilon_3(\phi)$")
    ax5.set_ylabel(r"$\epsilon_1^T(\phi)$")
    ax6.set_ylabel(r"$\epsilon_2^T(\phi)$")
    ax7.set_ylabel(r"$\epsilon_3^T(\phi)$")


    plt.tight_layout()
    plt.show()

In [ ]:
def essai1(N):
    n=4
    mu = 1e-2*M_pl
    M4=4*pi*pi*16/27
    plot_BI(M4,mu,n,N)
essai1(1)